# Assignment 01: Multi-class Classification 
In this Assignment, you will train a deep model on the CIFAR10 from the scratch using PyTorch.

### Basic Imports

In [1]:
import os
import time
import os.path as osp

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms
import torchvision

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from PIL import Image

### Hyperparameters

In [2]:
# random seed
SEED = 7
NUM_CLASS = 10

# Training
BATCH_SIZE = 128
NUM_EPOCHS = 50
EVAL_INTERVAL=1
SAVE_DIR = './log'

# Optimizer
LEARNING_RATE = 1e-1
MOMENTUM = 0.9
STEP=5
GAMMA=0.5

In [3]:
def FocalLoss(output, target, gamma, alpha=0.25):  
    # 根据模型的输出计算softmax，获得每个样本属于每个类别的概率  
    prob = F.softmax(output, dim=1)  
      
    # 根据one-hot形式的标签，找到每个样本对应的真实类别的概率  
    probs_one_hot = target * prob
    idces = probs_one_hot.nonzero()
    non_zero_entries = probs_one_hot[idces[:, 0], idces[:, 1]]
    prob_gt = non_zero_entries.view(-1) 
      
    # 计算每个样本的交叉熵损失  
    cross_entropy = -torch.log(prob_gt)  
      
    # 计算focal loss  
    focal_loss = alpha * (1 - prob_gt) ** gamma * cross_entropy  
      
    # 返回focal loss的均值  
    return focal_loss.mean()

In [4]:
# Loss functions
FocalLoss_2 = lambda output, target: FocalLoss(output, target, gamma = 2)
FocalLoss_05 = lambda output, target: FocalLoss(output, target, gamma = 0.5)
LOSS_Fs = [nn.CrossEntropyLoss(), 
           nn.L1Loss(), 
           FocalLoss_2, 
           FocalLoss_05]

### Device

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


### Dataset


In [6]:
# cifar10 transform
torch.manual_seed(SEED)

transform_cifar10_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_cifar10_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_set = torchvision.datasets.CIFAR10(root='../data', train=True,
                                        download=True, transform=transform_cifar10_train)
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root='../data', train=False,
                                       download=True, transform=transform_cifar10_test)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


Files already downloaded and verified
Files already downloaded and verified


### Model

In [7]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 4, 3)  
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(4, 8, 3)  
        self.fc1 = nn.Linear(8 * 6 * 6, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 8 * 6 * 6)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [8]:
model = ConvNet()
model.to(device)

ConvNet(
  (conv1): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=288, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)

### Task 1: per batch training/testing
---

Please denfine two function named ``train_batch`` and ``test_batch``. These functions are essential for training and evaluating machine learning models using batched data from dataloaders.

**To do**: 
1. Define the loss function i.e [nn.CrossEntropyLoss()](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html).
2. Take the image as the input and generate the output using the pre-defined SimpleNet.
3. Calculate the loss between the output and the corresponding label using the loss function.

In [9]:
# loss_f is the mean loss of a batch
def train_batch(model, loss_f, image, target):
    """
    Perform one training batch iteration.

    Args:
        model (torch.nn.Module): The machine learning model to train.
        image (torch.Tensor): Batch of input data (images).
        target (torch.Tensor): Batch of target labels.

    Returns:
        torch.Tensor: Model output (predictions) for the batch.
        torch.Tensor: Loss value calculated by the defined loss function loss_fn().
    """
    
    ##################### Write your answer here ##################
    output = model(image)
    target = F.one_hot(target, num_classes = NUM_CLASS).float()
    loss = loss_f(output, target)
    ###############################################################

    return output, loss

In [10]:
def test_batch(model, loss_f, image, target):
    """
    Perform one testing batch iteration.

    Args:
        model (torch.nn.Module): The machine learning model to evaluate.
        image (torch.Tensor): Batch of input data (images).
        target (torch.Tensor): Batch of target labels.

    Returns:
        torch.Tensor: Model output (predictions) for the batch.
        torch.Tensor: Loss value calculated for the batch.
    """

    ##################### Write your answer here ##################
    output = model(image)
    target = F.one_hot(target, num_classes = NUM_CLASS).float()
    loss = loss_f(output, target)
    ###############################################################

    return output, loss

### Transform the lables into one-hot form

### Model Training

In [11]:
def train_test_model(loss_f, learning_rate):

    training_loss = []
    training_acc = []
    testing_loss = []
    testing_acc = []
    
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=MOMENTUM)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP, gamma=GAMMA)

    for epoch in range(NUM_EPOCHS):
        model.train()
        torch.cuda.empty_cache()

        ##########################
        ### Training
        ##########################

        running_cls_loss = 0.0      # cummulated loss
        running_cls_corrects = 0

        for batch_idx, (image, target) in enumerate(train_dataloader):

            image = image.to(device)
            target = target.to(device)

            # train model
            outputs, loss = train_batch(model, loss_f, image, target)
            _, preds = torch.max(outputs, 1)


            loss_data = loss.data.item()
            if np.isnan(loss_data):
                raise ValueError('loss is nan while training')
            running_cls_loss += loss.item()
            running_cls_corrects += torch.sum(preds == target.data)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        epoch_loss = running_cls_loss / (len(train_set) / (BATCH_SIZE))
        epoch_acc = running_cls_corrects.double() / len(train_set)

        print(f'Epoch: {epoch+1}/{NUM_EPOCHS} Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        training_loss.append(epoch_loss)
        training_acc.append(epoch_acc.cpu().detach().numpy())

        # change learning rate
        scheduler.step()


        ##########################
        ### Testing
        ##########################
        # # eval model during training or in the last epoch
        if (epoch + 1) % EVAL_INTERVAL == 0 or (epoch +1) == NUM_EPOCHS:
            print('Begin test......')
            model.eval()

            val_loss = 0.0
            val_corrects = 0

            for batch_idx, (image, target) in enumerate(test_dataloader):

                image = image.to(device)
                target = target.to(device)

                # test model
                outputs, loss = test_batch(model, loss_f, image, target)
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item()
                val_corrects += torch.sum(preds == target.data)

            val_loss = val_loss / (len(test_set) / BATCH_SIZE)
            val_acc = val_corrects.double() / len(test_set)
            print(f'Test Loss: {val_loss:.4f} Acc: {val_acc:.4f}')
            testing_loss.append(val_loss)
            testing_acc.append(val_acc.cpu().detach().numpy())

            # save the model in last epoch
            if (epoch + 1) == NUM_EPOCHS:

                state = {
                'state_dict': model.state_dict(),
                'acc': epoch_acc,
                'epoch': (epoch+1),
                }

                # check the dir
                if not os.path.exists(SAVE_DIR):
                    os.makedirs(SAVE_DIR)

                # save the state
                torch.save(state, osp.join(SAVE_DIR, 'checkpoint_%s.pth' % (str(epoch+1))))
    
    loss_acc_df = pd.DataFrame({  
        'train_loss': training_loss,  
        'train_acc': training_acc,  
        'test_loss': testing_loss,  
        'test_acc': testing_acc  
    }) 
    
    return loss_acc_df

In [12]:
loss_acc_df0 = train_test_model(LOSS_Fs[0], LEARNING_RATE)

Epoch: 1/50 Train Loss: 2.0576 Acc: 0.2294
Begin test......
Test Loss: 1.8547 Acc: 0.3054
Epoch: 2/50 Train Loss: 1.9519 Acc: 0.2765
Begin test......
Test Loss: 1.8993 Acc: 0.3182
Epoch: 3/50 Train Loss: 1.9312 Acc: 0.2869
Begin test......
Test Loss: 1.7883 Acc: 0.3344
Epoch: 4/50 Train Loss: 1.9063 Acc: 0.2904
Begin test......
Test Loss: 1.8961 Acc: 0.2729
Epoch: 5/50 Train Loss: 1.8763 Acc: 0.2929
Begin test......
Test Loss: 1.8779 Acc: 0.3018
Epoch: 6/50 Train Loss: 1.7663 Acc: 0.3354
Begin test......
Test Loss: 1.7054 Acc: 0.3730
Epoch: 7/50 Train Loss: 1.7425 Acc: 0.3493
Begin test......
Test Loss: 1.6478 Acc: 0.3958
Epoch: 8/50 Train Loss: 1.7155 Acc: 0.3647
Begin test......
Test Loss: 1.6273 Acc: 0.4032
Epoch: 9/50 Train Loss: 1.7114 Acc: 0.3653
Begin test......
Test Loss: 1.7028 Acc: 0.3809
Epoch: 10/50 Train Loss: 1.6978 Acc: 0.3743
Begin test......
Test Loss: 1.6807 Acc: 0.3924
Epoch: 11/50 Train Loss: 1.6324 Acc: 0.3992
Begin test......
Test Loss: 1.5334 Acc: 0.4410
Epoch: 1

In [13]:
loss_acc_df1 = train_test_model(LOSS_Fs[1], LEARNING_RATE)

Epoch: 1/50 Train Loss: 0.1743 Acc: 0.1167
Begin test......
Test Loss: 0.1253 Acc: 0.1027
Epoch: 2/50 Train Loss: 0.1131 Acc: 0.1078
Begin test......
Test Loss: 0.1103 Acc: 0.1000
Epoch: 3/50 Train Loss: 0.1129 Acc: 0.1058
Begin test......
Test Loss: 0.1147 Acc: 0.1001
Epoch: 4/50 Train Loss: 0.1133 Acc: 0.1057
Begin test......
Test Loss: 0.1172 Acc: 0.1000
Epoch: 5/50 Train Loss: 0.1134 Acc: 0.1085
Begin test......
Test Loss: 0.1123 Acc: 0.1024
Epoch: 6/50 Train Loss: 0.1067 Acc: 0.1106
Begin test......
Test Loss: 0.1096 Acc: 0.1014
Epoch: 7/50 Train Loss: 0.1071 Acc: 0.1100
Begin test......
Test Loss: 0.1071 Acc: 0.1067
Epoch: 8/50 Train Loss: 0.1070 Acc: 0.1094
Begin test......
Test Loss: 0.1083 Acc: 0.1023
Epoch: 9/50 Train Loss: 0.1064 Acc: 0.1136
Begin test......
Test Loss: 0.1068 Acc: 0.1155
Epoch: 10/50 Train Loss: 0.1067 Acc: 0.1124
Begin test......
Test Loss: 0.1071 Acc: 0.1113
Epoch: 11/50 Train Loss: 0.1038 Acc: 0.1191
Begin test......
Test Loss: 0.1048 Acc: 0.1411
Epoch: 1

In [14]:
loss_acc_df2 = train_test_model(LOSS_Fs[2], LEARNING_RATE)

Epoch: 1/50 Train Loss: 0.3694 Acc: 0.2450
Begin test......
Test Loss: 0.3066 Acc: 0.3398
Epoch: 2/50 Train Loss: 0.3140 Acc: 0.3298
Begin test......
Test Loss: 0.2941 Acc: 0.3649
Epoch: 3/50 Train Loss: 0.3016 Acc: 0.3511
Begin test......
Test Loss: 0.2771 Acc: 0.3948
Epoch: 4/50 Train Loss: 0.2937 Acc: 0.3649
Begin test......
Test Loss: 0.2767 Acc: 0.4071
Epoch: 5/50 Train Loss: 0.2867 Acc: 0.3776
Begin test......
Test Loss: 0.2905 Acc: 0.3884
Epoch: 6/50 Train Loss: 0.2735 Acc: 0.4022
Begin test......
Test Loss: 0.2616 Acc: 0.4215
Epoch: 7/50 Train Loss: 0.2704 Acc: 0.4083
Begin test......
Test Loss: 0.2561 Acc: 0.4345
Epoch: 8/50 Train Loss: 0.2687 Acc: 0.4076
Begin test......
Test Loss: 0.2647 Acc: 0.4162
Epoch: 9/50 Train Loss: 0.2692 Acc: 0.4097
Begin test......
Test Loss: 0.2550 Acc: 0.4263
Epoch: 10/50 Train Loss: 0.2680 Acc: 0.4138
Begin test......
Test Loss: 0.2549 Acc: 0.4306
Epoch: 11/50 Train Loss: 0.2606 Acc: 0.4290
Begin test......
Test Loss: 0.2452 Acc: 0.4546
Epoch: 1

In [15]:
loss_acc_df3 = train_test_model(LOSS_Fs[3], LEARNING_RATE)

Epoch: 1/50 Train Loss: 0.3679 Acc: 0.3965
Begin test......
Test Loss: 0.3397 Acc: 0.4394
Epoch: 2/50 Train Loss: 0.3658 Acc: 0.3952
Begin test......
Test Loss: 0.3532 Acc: 0.4227
Epoch: 3/50 Train Loss: 0.3611 Acc: 0.4037
Begin test......
Test Loss: 0.3571 Acc: 0.4197
Epoch: 4/50 Train Loss: 0.3617 Acc: 0.4063
Begin test......
Test Loss: 0.3450 Acc: 0.4339
Epoch: 5/50 Train Loss: 0.3580 Acc: 0.4134
Begin test......
Test Loss: 0.3442 Acc: 0.4338
Epoch: 6/50 Train Loss: 0.3421 Acc: 0.4400
Begin test......
Test Loss: 0.3220 Acc: 0.4791
Epoch: 7/50 Train Loss: 0.3401 Acc: 0.4434
Begin test......
Test Loss: 0.3219 Acc: 0.4663
Epoch: 8/50 Train Loss: 0.3397 Acc: 0.4446
Begin test......
Test Loss: 0.3376 Acc: 0.4471
Epoch: 9/50 Train Loss: 0.3380 Acc: 0.4469
Begin test......
Test Loss: 0.3284 Acc: 0.4612
Epoch: 10/50 Train Loss: 0.3377 Acc: 0.4501
Begin test......
Test Loss: 0.3184 Acc: 0.4797
Epoch: 11/50 Train Loss: 0.3302 Acc: 0.4616
Begin test......
Test Loss: 0.3134 Acc: 0.4855
Epoch: 1

In [16]:
loss_acc_df0.columns = ["CE_train_l", "CE_train_a", "CE_test_l", "CE_test_a"]
loss_acc_df1.columns = ["L1_train_l", "L1_train_a", "L1_test_l", "L1_test_a"]
loss_acc_df2.columns = ["Focal_2_train_l", "Focal_2_train_a", "Focal_2_test_l", "Focal_2_test_a"]
loss_acc_df3.columns = ["Focal_05_train_l", "Focal_05_train_a", "Focal_05_test_l", "Focal_05_test_a"]
loss_acc_df = pd.concat([loss_acc_df0,
                         loss_acc_df1, 
                         loss_acc_df2, 
                         loss_acc_df3], axis=1)
loss_acc_df.to_csv('loss-acc-data/loss_acc.csv', index=False, encoding='utf-8')